# Monoids

A set S equipped with a binary operation $S \times S \to S$ is a monoid if it satisfies the following two axioms:

1. **Associativity**: $\forall a, b, c\in S\quad a(bc) = (ab)c$
2. **Identity element**: There exists a neutral element $e \in S$ such that for every element $a \in S$, the equalities $ea = ae = a$ hold.

```haskell
class Monoid a where
    mempty :: a
    mappend :: a -> a -> a
    
    mconcat :: [a] -> a
    mconcat = foldr mappend empty
```

Monoid laws:
```text
mempty `mappend` x = x; x `mappend` mempty = x
x `mappend` (y `mappend` z) = (x `mappend` y) `mappend` z
```

In [ ]:
trait Monoid[T] {
    def neutral: T
    def op(a: T, b: T): T
}

In [ ]:
val intSumMonoid: Monoid[Int] = new Monoid[Int] {
    def neutral: Int = 0
    def op(a: Int, b: Int): Int = a + b
} 

implicit val intProdMonoid: Monoid[Int] = new Monoid[Int] {
    def neutral: Int = 1
    def op(a: Int, b: Int): Int = a * b
}

implicit def listMonoid[A]: Monoid[List[A]] = new Monoid[List[A]] {
    def neutral: List[A] = Nil
    def op(xs: List[A], ys: List[A]): List[A] = xs ::: ys 
}

In [ ]:
def mappend[T: Monoid](xs: T, ys: T): T = 
    implicitly[Monoid[T]].op(xs, ys)


def mempty[T: Monoid](x: T): T = 
    implicitly[Monoid[T]].neutral

In [ ]:
println(mappend(List(1, 2), List(3)))

println(mappend(2, 3))

println(mempty(List("hello", "world")))

In [ ]:
def foldAll[T](xs: List[T])(implicit monoid: Monoid[T]): T =
    xs.foldLeft(monoid.neutral)(monoid.op)

In [ ]:
foldAll(List(1, 2, 3, 4))

foldAll(List(List(1, 2), Nil, List(3), List(4, 5)))

# Foldables

The main application of Monoids is to combine all values of a data structure to give a single value.

In [ ]:
// Foldable is a higher-kinded type (* -> *)

trait Foldable[F[_]] {
    def foldLeft[A, B](as: F[A])(init: B)(f: (B, A) => B): B
    
    // def foldRight[A, B](as: F[A])(init: B)(f: (A, B) => B): B
    // def concatenate[A](as: F[A])(m: Monoid[A]): A = foldLeft(as)(m.neutral)(m.op)
}

In [ ]:
implicit val listFold: Foldable[List] = new Foldable[List] {
    def foldLeft[A, B](as: List[A])(init: B)(f: (B, A) => B): B = as.foldLeft(init)(f)   
}

In [ ]:
def foldLeft[A, B, F[_]](xs: F[A])(init: B)(f: (B, A) => B)(implicit foldable: Foldable[F]): B = 
    foldable.foldLeft(xs)(init)(f)

In [ ]:
foldLeft(List(1, 2, 3))(0)(_ + _)

foldLeft(List("Hello", " ", "world!"))("")(_ + _)

In [ ]:
sealed trait BinaryTree[+T]

case object Leaf extends BinaryTree[Nothing]
case class Node[T](left: BinaryTree[T], value: T, right: BinaryTree[T]) extends BinaryTree[T]

In [ ]:
val tree: BinaryTree[Int] = Node(Node(Leaf, 1, Leaf), 10, Node(Leaf, 20, Node(Leaf, 30, Leaf)))

In [ ]:
implicit object TreeFoldable extends Foldable[BinaryTree] {
  def foldLeft[A, B](tree: BinaryTree[A])(init: B)(f: (B, A) => B): B = tree match {
      case Leaf => init
      case Node(left, value, right) =>
          val acc: B = foldLeft(left)(f(init, value))(f) 
          foldLeft(right)(acc)(f)
  }
}

In [ ]:
foldLeft(tree)(0)(_ + _)

foldLeft(tree)(1)(_ * _)